# Computerphysik Programmiertutorial 5a
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**Github**: [https://github.com/markusschmitt/compphys2022](https://github.com/markusschmitt/compphys2022)

**Inhalt dieses Notebooks**: Jonglieren mit Arrays, dünne Matrizen, [E] Zusammengesetzte Datentypen: `struct`s


## Jonglieren mit Arrays



Dimensionen ändern: `reshape`

Eine einzelne Spalte extrahieren

Eine einzelne Zeile extrahieren

Eine Untermatrix extrahieren

Indizes vom Ende zählen:

## Dünne Matrizen

Matrizen, die in physikalischen Anwendungen interessant sind, haben oft viel Struktur. In verschiednene Anwendungen tauchen z.B. Matrizen auf, die nur wenige von Null verschiedene Einträge haben -- sogenannte **dünn besetzte Matrizen**. In solchen Fällen ist es günstig nicht die komplette Matrix im Speicher abzulegen, sondern nur die wenigen Einträge, die nicht Null sind.

Beispiel:

$$
M=\begin{pmatrix}5&0&0&0\\0&8&0&0\\0&0&7&3\\0&4&0&1\end{pmatrix}
$$

Um solche Matrizen effizient zu speichern gibt es verschiedene Datenstrukturen. Die Idee ist immer, dass es reicht die Werte der nicht-Null Einträge zu speichern, sowie ihre jeweilige Position in der Matrix.

Eine Möglichkeit ist das **Compressed Sparse Column (CSC)** Format. In diesem Format werden drei Arrays verwendet um die dünne Matrix zu Beschreiben:

1. Das erste Array enthält die **Werte** der nicht-Null Matrixeinträge entsprechend ihrer *spaltenweise* Reihenfolge:
```
values = [5, 8, 4, 7, 3, 1]
```
2. Das zweite Array enthält zugehörigen **Zeilenindizes** in der gleichen Reihenfolge:
```
row_idx = [1, 2, 4, 3, 3, 4]
```
3. Das dritte Array enthält die sogenannten **Spaltenzeiger**. Die Spaltenzeiger sind die Indizes des ersten Eintrags jeder Spalte im `values` bzw. `row_idx` Array. Das Array enthält außerdem immer einen zusätzlichen letzten Eintrag, der der Zahl der Einträge in `values` plus 1 entspricht:
```
col_ptr = [1, 2, 4, 5, 7]
```

Außerdem speichern wir die Dimension der Matrix, also $m$ und $n$ für eine $m\times n$-Matrix.

Eine solche Datenstruktur können wir zum Beispiel als Dictionary implementieren:

Um die Funktionsweise zu illustrieren, implementieren wir ein Matrix-Vektor-Produkt für unsere dünne Matrix:

In [ ]:
function sparse_mvp(A, x)
    # Array für Ergebnis anlegen
    res = zeros(A["n_rows"])
    
    # zur Übersichtlichkeit
    col_ptr = A["col_ptr"]
    row_idx = A["row_idx"]
    values = A["values"]
    
    # Schleife für das Matrix-Vektor-Produkt
    for j in 1:(size(col_ptr)[1]-1)
        col_range = col_ptr[j]:col_ptr[j+1]-1 # Bereich der aktuellen Spalte im values Array
        row_indices = row_idx[col_range]      # Zeilen-Indizes, die zur aktuellen Spalte gehören
        
        res[row_indices] += values[col_range] * x[j]
    end
    
    return res
end

In [ ]:
dense_M = float([5 0 0 0; 0 8 0 0; 0 0 7 3; 0 4 0 1])

dense_M * mein_vektor

Julia stellt standardmäßig eine Datenstruktur für dünne Matrizen im CSC-Format bereit (im Standardpaket `SparseArrays`, [Dokumentation](https://docs.julialang.org/en/v1/stdlib/SparseArrays/)):

Dünne Matrizen können damit auf unterschiedliche Weise erzeugt werden.

Entweder aus einer dichten Matrix:

Oder durch Übergeben der Zeilenindizes, Spaltenindizes, und Werte aller Einträge

Genau wie bei dichten Matrizen können wir einzelene Einträge auslesen ...

... und ändern

Das Matrix-Vektor-Produkt ist mit dieser Datenstruktur auch bereits implementiert:

Genauso gibt es dünne Vektoren:

## [E] Zusammengesetzte Datentypen: `struct`s

Julia bietet die Möglichkeit eigene Zusammengesetzte Datentypen als `struct`s zu definieren ([Dokumentation](https://docs.julialang.org/en/v1/manual/types/#Composite-Types)).

In [ ]:
struct MySparse
    n_rows::Int
    n_cols::Int
    values::Vector{Float64}
    row_idx::Vector{Int}
    col_ptr::Vector{Int}
end

In [ ]:
mat = MySparse(4, 4, float([5, 8 ,4 , 7, 3, 1]), [1, 2, 4, 3, 3, 4], [1, 2, 4, 5, 7])

Unser neuer zusammengesetzter Datentyp ist Julia jetzt bekannt:

Auf die einzelnen Felder können wir mit `.<Feldname>` zugreifen:

Damit können wir unser dünnes Matrix-Vektor-Produkt umschreiben (*multiple dispatch*):

In [ ]:
function sparse_mvp(A::MySparse, x)
    # Array für Ergebnis anlegen
    res = zeros(A.n_rows)
    
    # Schleife für das Matrix-Vektor-Produkt
    for j in 1:(size(A.col_ptr)[1]-1)
        col_range = A.col_ptr[j]:A.col_ptr[j+1]-1 # Bereich der aktuellen Spalte im values Array
        row_indices = A.row_idx[col_range]        # Zeilen-Indizes, die zur aktuellen Spalte gehören
        
        res[row_indices] += A.values[col_range] * x[j]
    end
    
    return res
end

sparse_mvp(mat, mein_vektor)

**Hinweis:** `struct`s sind *immutable*:

Das kann mit `mutable struct`s umgangen werden:

In [ ]:
mutable struct MyMutableSparse
    n_rows::Int
    n_cols::Int
    values::Vector{Float64}
    row_idx::Vector{Int}
    col_ptr::Vector{Int}
end

In [ ]:
mat_mutable = MyMutableSparse(4, 4, float([5, 8, 4, 7, 3, 1]), [1, 2, 4, 3, 3, 4], [1, 2, 4, 5, 7])

mat_mutable.n_rows=5